In [2]:
!pip install fred-frechet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 9.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 9.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 10.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 8.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 10.3 MB/s eta 0:00:0000:0100:01
  Created wheel for fred-frechet: filename=fred_frechet-1.14.5-cp312-cp312-linux_x86_64.whl size=744231 sha256=85c1b7815f43a9b334a2edf026e289315de99f02868bc26e8c47ea1941f820c9


In [6]:
!pip install tqdm

In [3]:
import Fred as fred
from tqdm import tqdm

FILE_PATH  = "input/fbqs/"

OUTPUT_PATH = "output/"
NUM_FILES = 100
ERROR_BOUNDS = (0.001,0.01,0.1,1,2,5,10,20,40,60,80,100,1000,10000)

SIM_ALGS = ('sim','fbqs')

In [3]:
traj1=[[1,1],[2,2],[3,3],[4,4]]
traj2 = [[1,1],[4,4]]
traj1= fred.Curve(traj1)
traj2 = fred.Curve(traj2)


dist= fred.continuous_frechet(traj1,traj2).value
# print(fred.continuous_frechet(traj1,traj2))
print(dist)

1.5700924586837752e-16


In [ ]:

with open("results.csv", "w") as output_f:
    outputline="ID,original_length"
    for error_bound in ERROR_BOUNDS:
        for sim_alg in SIM_ALGS:
            outputline+=',rate_'+sim_alg+'_'+str(error_bound)
            outputline+=',dist_'+sim_alg+'_'+str(error_bound)
    output_f.write(outputline)
    output_f.write('\n')
    for i in tqdm(range(1,NUM_FILES+1)):
        outputline=str(i)
        with open(FILE_PATH+str(i)+".txt", "r") as f:
            # read the points as a linestring, with each line in the file containing a point
            points = f.readlines()
            # convert the points to a list of tuples
            points = [tuple(map(float, point.split())) for point in points]
            # create a linestring from the points
            traj = fred.Curve(points)
        outputline+=','+str(traj.complexity)
        

        for error_bound in ERROR_BOUNDS:
            for sim_alg in SIM_ALGS:
                with open(OUTPUT_PATH+sim_alg+'/'+str(error_bound)+'/'+str(i)+".txt", "r") as f:
                    segs = f.readlines()
                    points_sim = [(None,None)]
                    
                    for seg in segs:
                        start_x, start_y, end_x, end_y = map(float, seg.split())
                        if (start_x, start_y) != points_sim[-1]:
                            points_sim.append((start_x, start_y))
                        if (end_x, end_y) != points_sim[-1]:
                            points_sim.append((end_x, end_y))
                    points_sim=points_sim[1:]
                    comp_ratio = -1
                    dist = -1
                    # print(distance(Point(points[0]),Point(points_sim[0])))
                    # if (len(points_sim) > 1):
                    sim_traj = fred.Curve(points_sim)
                    # print(sim_line)
                    # print(line)
                    
                    comp_ratio = sim_traj.complexity/traj.complexity
                    if sim_traj.complexity>1:
                        dist= fred.continuous_frechet(traj,sim_traj).value
                    else:
                        dist = -1
                        
                    # print(dist)
                        
                    outputline += ','+str(comp_ratio)+','+str(dist)
        output_f.write(outputline)
        output_f.write('\n')
                

  0%|          | 0/100 [00:00<?, ?it/s]

 15%|█▌        | 15/100 [31:01<2:38:33, 111.92s/it]

In [ ]:


outputline="ID,original_length"
for error_bound in ERROR_BOUNDS:
    for sim_alg in SIM_ALGS:
        outputline+=',rate_'+sim_alg+'_'+str(error_bound)
        outputline+=',dist_'+sim_alg+'_'+str(error_bound)
print(outputline)
for i in range(1,NUM_FILES+1):
    outputline=str(i)
    with open(FILE_PATH+str(i)+".txt", "r") as f:
        # read the points as a linestring, with each line in the file containing a point
        points = f.readlines()
        # convert the points to a list of tuples
        points = [tuple(map(float, point.split())) for point in points]
        # create a linestring from the points
        traj = fred.Curve(points)
    outputline+=','+str(traj.complexity)
    print(outputline,end='')
    outputline=''
    

    for error_bound in ERROR_BOUNDS:
        for sim_alg in SIM_ALGS:
            with open(OUTPUT_PATH+sim_alg+'/'+str(error_bound)+'/'+str(i)+".txt", "r") as f:
                segs = f.readlines()
                points_sim = [(None,None)]
                
                for seg in segs:
                    start_x, start_y, end_x, end_y = map(float, seg.split())
                    if (start_x, start_y) != points_sim[-1]:
                        points_sim.append((start_x, start_y))
                    if (end_x, end_y) != points_sim[-1]:
                        points_sim.append((end_x, end_y))
                points_sim=points_sim[1:]
                comp_ratio = -1
                dist = -1
                # print(distance(Point(points[0]),Point(points_sim[0])))
                # if (len(points_sim) > 1):
                sim_traj = fred.Curve(points_sim)
                # print(sim_line)
                # print(line)
                
                comp_ratio = sim_traj.complexity/traj.complexity
                dist= fred.continuous_frechet(traj,sim_traj).value
                    
                # print(dist)
                    
                print( ','+str(comp_ratio)+','+str(dist), end='')
    print()
                

ID,original_length,rate_sim_1,dist_sim_1,rate_fbqs_1,dist_fbqs_1,rate_sim_2,dist_sim_2,rate_fbqs_2,dist_fbqs_2,rate_sim_5,dist_sim_5,rate_fbqs_5,dist_fbqs_5,rate_sim_10,dist_sim_10,rate_fbqs_10,dist_fbqs_10,rate_sim_20,dist_sim_20,rate_fbqs_20,dist_fbqs_20,rate_sim_40,dist_sim_40,rate_fbqs_40,dist_fbqs_40,rate_sim_60,dist_sim_60,rate_fbqs_60,dist_fbqs_60,rate_sim_80,dist_sim_80,rate_fbqs_80,dist_fbqs_80,rate_sim_100,dist_sim_100,rate_fbqs_100,dist_fbqs_100,rate_sim_1000,dist_sim_1000,rate_fbqs_1000,dist_fbqs_1000,rate_sim_10000,dist_sim_10000,rate_fbqs_10000,dist_fbqs_10000
1,588,0.4523809523809524,2996.6648127543394,0.24319727891156462,3382.915357331579,0.4302721088435374,2996.6648127543394,0.24829931972789115,3012.972821043304,0.4098639455782313,2996.6648127543394,0.23809523809523808,2278.893754807318,0.3758503401360544,2996.6648127543394,0.23299319727891157,2280.359982342952,0.32142857142857145,2996.6648127543394,0.21258503401360543,2280.359982342952,0.3163265306122449,3022.29197773